In [1]:
import io
import fitz
import re
from PIL import Image, ImageDraw, ImageFont, ImageColor
import operator
import pandas as pd 
import numpy as np
from datetime import datetime
import math
from tqdm import tqdm

# Ploting funcs

In [2]:
def plot_genus_blocks(page_df, draw, color = '#6c899e', w = 3):
    try:
        genus_list = page_df['draw_genus'].unique()
    except:
        #print("no GENUS found")
        return 

    for g in genus_list:
        temp_df = page_df[(page_df['draw_genus'] == g)]
        g_x0 = temp_df['x0'].min()
        g_y0 = temp_df['y0'].min()
        g_x1 = temp_df['x1'].max()
        g_y1 = temp_df['y1'].max()

        draw.rectangle((g_x0, g_y0, g_x1, g_y1), fill=None, outline=ImageColor.getrgb(color), width = w)
        
def plot_epithet_blocks(page_df, draw, color = '#660066', w = 3):
    try:
        epithet_list = page_df['draw_epithet'].unique()
    except:
        print("no EPITHET found")
        return 
    
    for e in epithet_list:
        temp_df = page_df[(page_df['draw_epithet'] == e)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_author_blocks(page_df, draw, color = '#a3a3a3', w = 1):
    try:
        author_list = page_df['draw_author'].unique()
    except:
        print("no AUTHOR found")
        return 

    for a in author_list:
        temp_df = page_df[(page_df['draw_author'] == a)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_infra_blocks(page_df, draw, color = '#ff6289', w = 1):
    try:
        infra_list = page_df['draw_infra'].unique()
    except:
        print("no INFRA Spp. found")
        return 

    for infra_spp in infra_list:
        temp_df = page_df[(page_df['draw_infra'] == infra_spp)]
        e_x0 = temp_df['x0'].min()
        e_y0 = temp_df['y0'].min()
        e_x1 = temp_df['x1'].max()
        e_y1 = temp_df['y1'].max()

        draw.rectangle((e_x0, e_y0, e_x1, e_y1), fill=None, outline=ImageColor.getrgb(color), width = w)

def plot_valid_words(page_df, draw, color = '#660044', w = 2):
    blocks = page_df['block_no'].unique()
    """for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            words = page_df[cond]['word_no'].unique()
            page_df = page_df.copy()
            for w in words:
                x0 = page_df[(cond) & (page_df['word_no'] == w)]['x0'].item()
                y0 = page_df[(cond) & (page_df['word_no'] == w)]['y0'].item()
                x1 = page_df[(cond) & (page_df['word_no'] == w)]['x1'].item()
                y1 = page_df[(cond) & (page_df['word_no'] == w)]['y1'].item()
                draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb(color), width = w)
    """
    for index, row in page_df.iterrows():
        x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1'] 
        draw.rectangle((x0, y0, x1, y1), fill=None, outline=ImageColor.getrgb(color), width = w)

# Import Vol1 Index

In [3]:
#pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf"
pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"
index = range(616, 639)
doc = fitz.open(pdf_dir)
pages = [doc[i] for i in range(doc.page_count)] #doesn't work anymore? [doc[i] for i in range(doc.pageCount)]
#index = list(range(555, 583))

pdf_dir = "../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf"
index = range(616, 639)

TARGET_DPI = 300
mat = fitz.Matrix(TARGET_DPI/ 72, TARGET_DPI/ 72)

indent_groups = []
indent_err = 15

# regex based boolean functions

In [4]:
def valid(word):
    """
    valid words are words that are:
    - at least 2 characters
        - unless it's x (symbol for hybrid)
    """
    return (not bool(re.search(r"[0-9]+[,.]?", word))) and \
            (word != 'NOUVELLE' and word != 'FLORE') and \
            (len(word) > 1 or \
                word == 'x' or word == 'X' or word == '×' or word == r'\u00D7') and \
            ''.join(e for e in word if e.isalpha()).isalpha()
    
def is_genus(word):
    """
    A word in the index might be a genus if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - first letter upper case
        - all but first lowecase 
    in regex: ^[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+$ #ignoring strict beggining and end cause of noise
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"[A-ZÀÂÄÈÉÊËÎÏÔŒÙÛÜŸÇ\u00D7]{1}[a-zàâäèéêëîïôœùûüÿç]*[-]?[a-zàâäèéêëîïôœùûüÿç]+"
    return re.search(regex, word)
    

def is_epithet(word):
    """
    A word in the index might be an epithet if it satisfies the following properties:
    - letters: french alphabet + at most one hyphen (which is not first or last letter)
        - all letters lowecase 
    in regex: ^[a-zàâäèéêëîïôœùûüÿç]+[-]?[a-zàâäèéêëîïôœùûüÿç]+$ #ignoring strict beggining and end cause of noise 
        * based on the current expression it'd also be at least 2 letters long
    """
    regex = r"[a-zàâäèéêëîïôœùûüÿç\u00D7]+[-]?[a-zàâäèéêëîïôœùûüÿç]+"
    return re.search(regex, word)
    
def is_hybrid(word):
    regex = r"^(([Xx\u00D7])|([Xx\u00D7]\.))$"
    return re.search(regex, word)

def is_infra(word):
    regex = r"^(var\.)|(subsp\.)"
    return re.search(regex, word)

# pre-processing func

In [142]:
def preprocessing(page_num, indent_err = 30):
    
    #initiate dataframe
    page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    
    #to get blocks, lines, span etc
    """blocks = page.get_text("dict")["blocks"]
    for b in blocks:
        if "lines" in b:
            for l in b["lines"]:
                for span in b["lines"]:
                    for word in span["spans"][0]["text"].split():
                        print("word:", word, "\t\t\t flags:", span["spans"][0]["flags"], "\t\t\t font:", span["spans"][0]["font"])"""

    #idea: get this and get coordinates, words, block_no_etc based on this.
    #or perhaps create a db from the dictionary of page.get_text("dict")and the create rows for each text line by splitting it:
    #based on this: https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

    #add page number to dataframe
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    #initiate all columns that will be added
    page_df['genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['taxon rank'] = np.array([np.NaN]*page_df.shape[0])
    page_df['error_check'] = np.array([np.NaN]*page_df.shape[0])
    page_df['flags'] = np.array([np.NaN]*page_df.shape[0])
    page_df['font'] = np.array([np.NaN]*page_df.shape[0])
    #updating coordinates to represent target DPI
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    #get x corner coordinates 
    x_min = page_df['x0'].min()
    x_max = page_df['x1'].max()

    y_max = page_df['y1'].max()

    #Remove the extra flore - 18 at page 545
    if page_num == index[4]:
        page_df = page_df[~((page_df["word"] == 'Flore') & (page_df['y1'] == y_max))]
    #invalid words dataframe -- for error checking
    pruned_words_df = page_df[~page_df["word"].apply(valid)].reset_index()
    #prune out invalid words (based on function valid)
    #page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    indent_groups = []
    blocks = page_df['block_no'].unique()
    for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            #reset word_no values (useful for cases where word that was originally at 0th index was pruned out)
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            num_words = len(page_df[cond]['word_no'])
            page_df.loc[cond, 'word_no'] = np.arange(num_words).astype(int) #this is slowww
            #set column number (0 or 1)
            x_0 = page_df[cond]['x0'].min()
            #THIS DOESN'T WORK AAAA -- issue was with line no thing
            if not np.isnan(x_0):
                page_df.loc[cond, 'col_no'] = np.array([int(x_0 > ((x_min + x_max) / 2))]*num_words).astype(int)

                #initiate indent groups -- only first word should get an indent_group value 
                new_group = True
                for g_i in range(len(indent_groups)):
                    g = indent_groups[g_i]
                    g_arr = np.array(g)
                    if x_0 <= np.mean(g_arr) + indent_err and x_0 >= np.mean(g_arr) - indent_err:
                        g.append(x_0)
                        new_group = False
                        page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
                if new_group:
                    indent_groups.append([x_0])
                    g_i = len(indent_groups) - 1
                    page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)

    #print("indent groups:", indent_groups)
    #return updated page_df, pruned_words_df, indent groups
    return page_df.reset_index(), pruned_words_df, indent_groups

#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe

In [143]:
def preprocessing2(page_num, indent_err = 30):
    
    #initiate dataframe
    page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    
    #to get blocks, lines, span etc
    """blocks = page.get_text("dict")["blocks"]
    for b in blocks:
        if "lines" in b:
            for l in b["lines"]:
                for span in b["lines"]:
                    for word in span["spans"][0]["text"].split():
                        print("word:", word, "\t\t\t flags:", span["spans"][0]["flags"], "\t\t\t font:", span["spans"][0]["font"])"""

    #idea: get this and get coordinates, words, block_no_etc based on this.
    #or perhaps create a db from the dictionary of page.get_text("dict")and the create rows for each text line by splitting it:
    #based on this: https://stackoverflow.com/questions/12680754/split-explode-pandas-dataframe-string-entry-to-separate-rows

    #add page number to dataframe
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    #initiate all columns that will be added
    page_df['genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['taxon rank'] = np.array([np.NaN]*page_df.shape[0])
    page_df['error_check'] = np.array([np.NaN]*page_df.shape[0])
    page_df['flags'] = np.array([np.NaN]*page_df.shape[0])
    page_df['font'] = np.array([np.NaN]*page_df.shape[0])
    #updating coordinates to represent target DPI
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    #get x corner coordinates 
    x_min = page_df['x0'].min()
    x_max = page_df['x1'].max()

    y_max = page_df['y1'].max()

    #Remove the extra flore - 18 at page 545
    if page_num == index[4]:
        page_df = page_df[~((page_df["word"] == 'Flore') & (page_df['y1'] == y_max))]
    #invalid words dataframe -- for error checking
    pruned_words_df = page_df[~page_df["word"].apply(valid)].reset_index()
    #prune out invalid words (based on function valid)
    #page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    indent_groups = []
    blocks = page_df['block_no'].unique()
    for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            #reset word_no values (useful for cases where word that was originally at 0th index was pruned out)
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            num_words = len(page_df[cond]['word_no'])
            page_df.loc[cond, 'word_no'] = np.arange(num_words).astype(int) #this is slowww
            #set column number (0 or 1)
            x_0 = page_df[cond]['x0'].min()
            #THIS DOESN'T WORK AAAA -- issue was with line no thing
            if not np.isnan(x_0):
                page_df.loc[cond, 'col_no'] = np.array([int(x_0 > ((x_min + x_max) / 2))]*num_words).astype(int)

                #initiate indent groups -- only first word should get an indent_group value 
                new_group = True
                for g_i in range(len(indent_groups)):
                    g = indent_groups[g_i]
                    g_arr = np.array(g)
                    if x_0 <= np.mean(g_arr) + indent_err and x_0 >= np.mean(g_arr) - indent_err:
                        g.append(x_0)
                        new_group = False
                        page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
                if new_group:
                    indent_groups.append([x_0])
                    g_i = len(indent_groups) - 1
                    page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)

    #print("indent groups:", indent_groups)
    #return updated page_df, pruned_words_df, indent groups
    return page_df.reset_index(), pruned_words_df, indent_groups

#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe

In [184]:
page_df = pd.DataFrame(pages[page_num-10].get_text("dict")['blocks'])
page_df


,number,type,bbox,width,height,ext,colorspace,xres,yres,bpc,transform,size,image,lines
0,0,1,"(231.0279998779297, 31.800006866455078, 258.14...",113.0,23.0,png,1.0,96.0,96.0,1.0,"(27.1200008392334, 0.0, -0.0, 5.51999998092651...",1633.0,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,NaN
1,1,0,"(180.0, 28.90131950378418, 410.50201416015625,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 8.954887390136719, 'flags..."
2,2,1,"(40.30799865722656, 56.52001190185547, 71.9879...",132.0,26.0,png,1.0,96.0,96.0,1.0,"(31.68000030517578, 0.0, -0.0, 6.2399997711181...",1633.0,"b""\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...",NaN
3,3,0,"(39.84000015258789, 54.09002685546875, 123.071...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 9.27043628692627, 'flags'..."
4,4,1,"(26.312999725341797, 63.23997116088867, 57.993...",132.0,27.0,png,1.0,96.0,96.0,1.0,"(31.68000030517578, 0.0, -0.0, 6.4800000190734...",1633.0,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,199,0,"(378.9599914550781, 554.72998046875, 411.70111...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 9.22225570678711, 'flags'..."
200,200,0,"(356.8800048828125, 573.4500122070312, 384.379...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 9.22225570678711, 'flags'..."
201,201,0,"(379.44000244140625, 583.0499877929688, 411.72...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 9.22225570678711, 'flags'..."
202,202,0,"(397.67999267578125, 592.6500244140625, 411.37...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'spans': [{'size': 8.954887390136719, 'flags..."


In [265]:
line_dicts = page_df[~page_df['lines'].isnull()].explode('lines', ignore_index=False)
list(line_dicts['lines'])
"""
pd.DataFrame.from_dict(
    {(i, j): user_dict[i][j] for i in user_dict.keys() for j in user_dict[i].keys()},
    orient="index",
)""";
df = pd.DataFrame(list((pd.DataFrame(list(line_dicts['lines']))['spans'].explode('spans'))))
df['text'] = df['text'].apply(lambda x : list(x.split()))
df = df.explode('text')
#df['word_no'] = df['text'].apply(lambda x : x[0])
#df['word'] = df['text'].apply(lambda x : x[1])
#df[df['word'].apply(lambda x : x.isalpha())]

split_df = pd.DataFrame(df['bbox'].tolist(), columns=['x0', 'y0', 'x1', 'y1'])
# concat df and split_df
df = pd.concat([df.reset_index(), split_df], axis=1)

#df['bbox']
#[['flags', 'font']]
#Ideas to fix the rest of this: 
#get line number and block number similarly
#okay the numbering seems to get messed up this way yikes? okay need to add position on spans and that should fix it!d

page_df_old = pd.DataFrame(pages[page_num-10].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
page_df_old.head()
df.head()

,index,size,flags,font,color,ascender,descender,text,origin,bbox,x0,y0,x1,y1
0,0,8.954887,4,Times-Roman,0,0.75000,-0.25000,NOUVELLE,"(180.0, 37.32000732421875)","(180.0, 30.57000732421875, 225.40687561035156,...",180.000000,30.570007,225.406876,39.570007
1,1,9.000000,4,Times-Roman,0,0.75000,-0.25000,FLORE,"(225.40829467773438, 37.32000732421875)","(225.40829467773438, 30.57000732421875, 258.44...",225.408295,30.570007,258.445862,39.570007
2,2,9.947885,4,Helvetica,0,0.77002,-0.22998,553,"(395.0400085449219, 36.5999755859375)","(395.0400085449219, 28.90131950378418, 410.502...",395.040009,28.901320,410.502014,38.899319
3,3,9.270436,4,Times-Roman,0,0.75000,-0.25000,scoparia,"(39.84000015258789, 60.84002685546875)","(39.84000015258789, 54.09002685546875, 71.5518...",39.840000,54.090027,71.551834,63.090027
4,4,9.222256,4,Times-Roman,0,0.75000,-0.25000,(L.),"(71.55329132080078, 60.84002685546875)","(71.55329132080078, 54.09002685546875, 123.071...",71.553291,54.090027,123.071503,63.090027


In [278]:
joined_df = pd.concat([df, page_df_old], axis=1, join="outer")
assert joined_df[joined_df['text'] != joined_df['word']].empty
joined_df = joined_df[['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no', 'flags', 'font', 'size', 'color']]

In [252]:
df.head()
(df['in_x0'] <= page_df_old['in_x0']) & (page_df_old['in_x0'] <= df['in_x1']) & \
    (df['in_y0'] <= page_df_old['in_y0']) & (page_df_old['in_y0'] <= df['in_y1'])

ValueError: Can only compare identically-labeled Series objects

In [112]:
a = preprocessing(2, indent_err = 30)
pages[page_num]

import json
page = doc.load_page(page_num)
"""blocks = json.loads(page)["blocks"]
for block in blocks:
    if "lines" in block:
        for line in block["lines"]:
            line_text = ""
            for span in block["lines"]:
                line_text += span["spans"][0]["text"].encode("utf-8")
            if last_text != line_text:
                is_recording, main_class = add_line(line_text, is_recording, main_class)
                last_text = line_text""";
#page.get_text("text", flags=8)
#page.get_text_blocks()

page_display = page.get_displaylist()
dictionary_elements = page_display.get_textpage().extractDICT()
for block in dictionary_elements['blocks']:
    for line in block['lines']:
        #print(block)
        line_text = ''
        #print(line)
        #break
        for span in line['spans']:
            #if span['font']..lower()
            line_text += ' ' + span['text']
            #print(line_text)
        #print("NEW LINE")


#{'number': 0, 'type': 0, 'bbox': (166.8000030517578, 26.6500244140625, 399.86077880859375, 35.6500244140625), 'lines': [{'spans': [{'size': 8.954887390136719, 'flags': 4, 'font': 'Times-Roman', 'color': 0, 'ascender': 0.75, 'descender': -0.25, 'text': 'NOUVELLE FLORE ', 'origin': (166.8000030517578, 33.4000244140625), 'bbox': (166.8000030517578, 26.6500244140625, 247.62559509277344, 35.6500244140625)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (166.8000030517578, 26.6500244140625, 247.62559509277344, 35.6500244140625)}, {'spans': [{'size': 8.954887390136719, 'flags': 4, 'font': 'Times-Roman', 'color': 0, 'ascender': 0.75, 'descender': -0.25, 'text': '563', 'origin': (384.6168518066406, 33.4000244140625), 'bbox': (384.6168518066406, 26.6500244140625, 399.86077880859375, 35.6500244140625)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (384.6168518066406, 26.6500244140625, 399.86077880859375, 35.6500244140625)}]}
#{'spans': [{'size': 8.954887390136719, 'flags': 4, 'font': 'Times-Roman', 'color': 0, 'ascender': 0.75, 'descender': -0.25, 'text': 'NOUVELLE FLORE ', 'origin': (166.8000030517578, 33.4000244140625), 'bbox': (166.8000030517578, 26.6500244140625, 247.62559509277344, 35.6500244140625)}], 'wmode': 0, 'dir': (1.0, 0.0), 'bbox': (166.8000030517578, 26.6500244140625, 247.62559509277344, 35.6500244140625)}

TypeError: can only concatenate str (not "bytes") to str

In [128]:
page.get_text("words");
for b in page.get_text("dict")["blocks"]:
    if 'lines' in b:
        b['lines']

In [139]:
#page = doc.getPageText(2, output="json")
page = doc.load_page(page_num-4)

blocks = page.get_text("dict")["blocks"]
for b in blocks:
    if "lines" in b:
        for l in b["lines"]:
            for span in b["lines"]:
                for word in span["spans"][0]["text"].split():
                    print("word:", word, "\t\t\t flags:", span["spans"][0]["flags"], "\t\t\t font:", span["spans"][0]["font"])

word: NOUVELLE 			 flags: 20 			 font: Times-Bold
word: FLORE 			 flags: 20 			 font: Times-Bold
word: 559 			 flags: 4 			 font: Times-Roman
word: NOUVELLE 			 flags: 20 			 font: Times-Bold
word: FLORE 			 flags: 20 			 font: Times-Bold
word: 559 			 flags: 4 			 font: Times-Roman
word: Sagina 			 flags: 20 			 font: Times-Bold
word: apetala 			 flags: 4 			 font: Times-Roman
word: L. 			 flags: 4 			 font: Times-Roman
word: libanotica 			 flags: 4 			 font: Times-Roman
word: Rech. 			 flags: 4 			 font: Times-Roman
word: fil. 			 flags: 4 			 font: Times-Roman
word: maritima 			 flags: 4 			 font: Times-Roman
word: G. 			 flags: 4 			 font: Times-Roman
word: Don. 			 flags: 4 			 font: Times-Roman
word: octandra 			 flags: 6 			 font: Times-Italic
word: apetala 			 flags: 4 			 font: Times-Roman
word: L. 			 flags: 4 			 font: Times-Roman
word: libanotica 			 flags: 4 			 font: Times-Roman
word: Rech. 			 flags: 4 			 font: Times-Roman
word: fil. 			 flags: 4 			 font: Times-Roman
w

# Finding indentations associated with genus, epithet, infra

In [7]:
types = ['genus', 'epithet', 'infra', 'author', 'misc.']
def n_leftmost_indent(df, n):
    """return a tuple with at most 3 elements each element itself is a tuple containing indent group, mean, group len"""
    indent_groups = [(g, df[(df['indent_group'] == g) & (df['word_no'] == 0)]['x0'].mean(), len(df[(df['indent_group'] == g) & (df['word_no'] == 0)]['x0'])) for g in df['indent_group'].unique()]
    indent_groups.sort(key = lambda x : x[1])
    #print(indent_groups[:n])
    return indent_groups[:n]

In [8]:
def get_genusEpithetInfra_indent(col_df):
    leftmost_3_indents = n_leftmost_indent(col_df, 2) #for vol1 only 2 indentations will be given 
    min_gap = 0
    max_gap = 75 #error is 30 -- less than 50% of max gap (which will be ignored for now)

    # possibly not specific enough
    # first identifying indent based don distance from one another only
    """if len(leftmost_3_indents) == 3:
        if leftmost_3_indents[0][1] < max_gap:
            leftmost_3_indents = leftmost_3_indents[1:]
        elif ((leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) > max_gap or \
            (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) < min_gap): #comparing first two (if satisfied last two will be checked in next if block)
            leftmost_3_indents = [max(leftmost_3_indents[1:], key = lambda x : x[2])] + [leftmost_3_indents[2]]
        elif (leftmost_3_indents[2][1] - leftmost_3_indents[1][1]) > max_gap or \
            (leftmost_3_indents[2][1] - leftmost_3_indents[1][1]) < min_gap: #comparing last two
            leftmost_3_indents = [leftmost_3_indents[0]] + [max(leftmost_3_indents[1:], key = lambda x : x[2])]

    if len(leftmost_3_indents) == 2:
        if leftmost_3_indents[0][1] < max_gap:
            leftmost_3_indents = leftmost_3_indents[1]
        elif (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) > max_gap or (leftmost_3_indents[1][1] - leftmost_3_indents[0][1]) < min_gap:
            leftmost_3_indents = [max(leftmost_3_indents, key = lambda x : x[2])]"""

    has_genus, has_epithet, has_infra = False, False, False
    genus_indent, epithet_indent, infra_indent = -1, -1, -1
    if len(leftmost_3_indents) == 3 and type(leftmost_3_indents) == type([1,2,3]):
        has_genus, has_epithet, has_infra = True, True, True
        print("leftmost 3:", leftmost_3_indents)
        genus_indent, epithet_indent, infra_indent = [el[0] for el in leftmost_3_indents]
    elif len(leftmost_3_indents) == 2:
        if col_df[col_df['indent_group'] == leftmost_3_indents[1][0]]['word'].apply(is_infra).any():
            has_genus, has_epithet, has_infra = False, True, True
            epithet_indent, infra_indent = [el[0] for el in leftmost_3_indents]
        else:
            has_genus, has_epithet, has_infra = True, True, False
            genus_indent, epithet_indent = [el[0] for el in leftmost_3_indents]
    elif len(leftmost_3_indents) == 1 or type(leftmost_3_indents) == type((1,2,3)): 
        if type(leftmost_3_indents) == type((1,2,3)):
            leftmost_3_indents = [leftmost_3_indents]
        has_genus, has_epithet, has_infra = False, True, False
        epithet_indent = leftmost_3_indents[0][0]

    return genus_indent, epithet_indent, infra_indent, leftmost_3_indents

# Processing column dataframes


In [9]:
def process_col(col_df, genus, epithet, draw_genus, draw_epithet, draw_infra = np.NaN):
    genus_indent, epithet_indent, infra_indent, indent_3_left = get_genusEpithetInfra_indent(col_df)
    #print(genus_indent, epithet_indent, infra_indent, indent_3_left)
    
    blocks = col_df['block_no'].unique()
    start_word_cond = -1 
    author = ''
    #draw_infra = np.NaN
    
    col_df = col_df.copy()
    for index, row in col_df.iterrows():
        b, l, w = row['block_no'], row['line_no'], row['word_no']
        word, indent_group = row['word'], row['indent_group']
        row_cond = (col_df['line_no'] == l) & (col_df['block_no'] == b) & (col_df['word_no'] == w) 
        process_hybrid = False
        process_infra = False
        if w == 0: 
            start_word_cond = row_cond
            if indent_group == genus_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                genus = word
                draw_genus = genus
                epithet = ''
                draw_epithet = ''
                author = ''
                misc = ''
                infra = ''
                col_df.loc[start_word_cond, 'genus'] = genus
                col_df.loc[start_word_cond, 'taxon rank'] = 'genus'
                if not is_genus(word):
                    col_df.loc[row_cond, 'error_check'] = True
                col_df.loc[row_cond, 'draw_genus'] = draw_genus
                col_df.loc[row_cond, 'author'] = ''

            elif indent_group == epithet_indent and not ''.join(e for e in word if e.isalpha()).isupper():
                epithet = word
                author = ''
                col_df.loc[row_cond, 'genus'] = genus
                col_df.loc[row_cond, 'epithet'] = epithet
                col_df.loc[row_cond, 'taxon rank'] = 'species'
                if not is_epithet(word):
                    col_df.loc[row_cond, 'error_check'] = True
                draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(b) + '_' + str(l)
                col_df.loc[row_cond, 'draw_genus'] = draw_genus
                col_df.loc[row_cond, 'draw_epithet'] = draw_epithet
                col_df.loc[row_cond, 'author'] = ''
        
        else:
            #print(genus, epithet)
            if w == 1 and epithet == '': 
                epithet = word
                misc = ''
                infra = ''
                author = ''
                start_word_cond = row_cond
                col_df.loc[row_cond, 'genus'] = genus
                col_df.loc[row_cond, 'epithet'] = epithet
                col_df.loc[row_cond, 'taxon rank'] = 'species'
                if not is_epithet(word):
                    col_df.loc[row_cond, 'error_check'] = True
                draw_epithet = str(genus) + '_' + str(epithet) +'_' + str(b) + '_' + str(l)
                col_df.loc[row_cond, 'draw_genus'] = draw_genus
                col_df.loc[row_cond, 'draw_epithet'] = draw_epithet
                col_df.loc[row_cond, 'author'] = ''
            elif (type(genus) == type("STR") and genus != '') or (type(epithet) == type("STR") and epithet != ''):
                #print(col_df.loc[start_word_cond, 'author'])
                """if np.isnan(col_df.loc[start_word_cond, 'author'].item()):
                    author == ''
                    col_df.loc[start_word_cond, 'author'] = ''"""
                curr_author_part = word + ' '
                col_df.loc[start_word_cond, 'author'] += curr_author_part
                col_df.loc[row_cond, 'draw_author'] = 'author_'+str(b)+'_'+str(l)
                col_df.loc[row_cond, 'draw_genus'] = draw_genus
            #col_df.loc[word_cond, 'draw_genus'] = draw_genus
            #if epithet:
            #    col_df.loc[word_cond, 'draw_epithet'] = draw_epithet
            #if infra: 
            #    col_df.loc[word_cond, 'draw_infra'] = draw_infra"""

    #Last author
    """if author != '':
        col_df.loc[start_word_cond, 'author'] = author"""
                    

    return col_df, genus, epithet, draw_genus, draw_epithet


# Run PreProcessing

In [10]:
#preprocessing
genus = np.NaN
df_dict = {}
pruned_dict = {}

for page_num in tqdm(index):
    page_df, pruned_df, indent_group = preprocessing(page_num)
    df_dict[page_num] = page_df
    pruned_dict[page_num] = pruned_df

genus = np.NaN
epithet = np.NaN
draw_genus = np.NaN
draw_epithet = np.NaN
result_ims_valid_words = []
df_list = []

for page_num in tqdm(index):
    #page_num = index[-1]
    #process the pre-processed dfs
    page_df = df_dict[page_num]
    
    #for drawing
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)
    
    plot_valid_words(page_df, draw, color = '#660044', w = 2)
    result_ims_valid_words.append(image)
    
    #break 
#result_ims_valid_words[0].save(OUTPUT_PATH + "preprocessed/" + 'valid_words' + TAIL_STR + '.pdf',save_all=True, append_images=result_ims[1:])

100%|██████████| 23/23 [00:06<00:00,  3.47it/s]


In [11]:
df = pd.concat(df_dict, axis = 0)

In [12]:
df[(df['page_num'] == 627) & (df['block_no'] > 9)].head(50)

index       in_x0       in_y0       in_x1       in_y1  \
627 28     28   47.759998  181.994995   78.790741  190.694992   
    29     29   83.194374  181.994995  106.577103  190.694992   
    30     30   48.000000  191.595001  100.472343  200.294998   
    31     31  105.412971  191.595001  132.389404  200.294998   
    32     32   47.759998  200.955017   81.761902  209.655014   
    33     33   83.211807  200.955017  105.130463  209.655014   
    34     34   47.520000  211.274994   96.020859  219.974991   
    35     35   97.470764  211.274994  119.583961  219.974991   
    36     36   48.000000  220.875000   86.440514  229.574997   
    37     37   91.054565  220.875000  118.230095  229.574997   
    38     38   47.759998  229.994995   86.656219  238.694992   
    39     39   90.245354  229.994995  108.242897  238.694992   
    40     40  112.342682  229.994995  118.977386  238.694992   
    41     41  122.310097  229.994995  139.112305  238.694992   
    42     42   48.000000  239.355011   81.121414  248.055008   
    43     43   85.493217  239.355011  112.470299  248.055008   
    44     44   47.759998  248.955017   98.137878  257.654999   
    45     45  102.799850  248.955017  124.496681  257.654999   
    46     46   47.520000  258.795013   84.357697  267.494995   
    47     47   88.317200  258.795013  111.999062  267.494995   
    48     48   47.759998  268.154999   94.718056  276.854980   
    49     49   96.166946  268.154999  104.054520  276.854980   
    50     50   47.520000  277.755005  111.317520  286.454987   
    51     51  115.561569  277.755005  147.628021  286.454987   
    52     52  151.869980  277.755005  175.361862  286.454987   
    53     53   47.279999  285.776703   79.099052  296.921387   
    54     54   82.269852  285.776703  110.349594  296.921387   
    55     55   47.279999  295.110596   82.219322  305.942108   
    56     56   83.669456  295.136719  110.743759  306.281403   
    57     57   47.279999  304.496704   73.083153  315.641388   
    58     58   76.864777  304.496704  104.345589  315.641388   
    59     59   47.040001  314.070618   77.450432  324.902100   
    60     60   78.900665  314.096710  102.481468  325.241394   
    61     61   47.040001  324.176697   90.653366  335.321381   
    62     62   93.118011  325.994995  116.799957  334.694977   
    63     63   46.799999  335.115021   72.903305  343.815002   
    64     64   76.640617  335.115021   98.254822  343.815002   
    65     65  102.564034  335.115021  109.105484  343.815002   
    66     66  112.554550  335.115021  147.270111  343.815002   
    67     67   47.040001  344.714996   90.040993  353.414978   
    68     68   94.975563  344.714996  116.586609  353.414978   
    69     69   46.799999  354.315002   90.752556  363.014984   
    70     70   92.201180  354.315002  119.529755  363.014984   
    71     71   46.799999  363.915009   76.041847  372.614990   
    72     72   80.036499  363.915009  100.950371  372.614990   
    73     73   46.799999  373.515015   78.759041  382.214996   
    74     74   80.209106  373.515015  107.537971  382.214996   
    75     75   46.560001  383.115021   93.729263  391.815002   
    76     76   97.318390  383.115021  118.733002  391.815002   
    77     77  122.833679  383.115021  129.479675  391.815002   

                     word  block_no  line_no  word_no  page_num  ...  infra  \
627 28           albicans        10        0        0       627  ...    NaN   
    29              Lange        10        0        1       627  ...    NaN   
    30     antilibanotica        10        1        0       627  ...    NaN   
    31             Dinsm.        10        1        1       627  ...    NaN   
    32          atrofusca        10        2        0       627  ...    NaN   
    33              Baker        10        2        1       627  ...    NaN   
    34       atropurpurea        10        3        0       627  ...    NaN   
    35              Baker        10        3        1       

# Final Results + SAVE

In [13]:
#Setting up files and directories for saving the results
SCRIPT_NAME = "reverted_index_Vol1_synonyms_pruned.ipynb"
SCRIPT_OUTPUT_PATH = "../output/index/" + SCRIPT_NAME + "/"
DATE_STR = datetime.now().strftime("%Y_%m_%d") 
TIME_STR = datetime.now().strftime("%H%M")
QUICK_FIX = False
TAIL_STR = ''

if QUICK_FIX:
    OUTPUT_PATH = SCRIPT_OUTPUT_PATH + DATE_STR + "/QuickFix/" 
    #TAIL_STR = '_' + DATE_STR + '_' + TIME_STR
else:
    OUTPUT_PATH = SCRIPT_OUTPUT_PATH + DATE_STR + "/" + TIME_STR + "/"

try:
    os.makedirs(OUTPUT_PATH)
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(OUTPUT_PATH + "preprocessed/")
except FileExistsError:
    # directory already exists
    pass

try:
    os.makedirs(OUTPUT_PATH + 'raw/')
except FileExistsError:
    # directory already exists
    pass

In [14]:
genus = np.NaN
epithet = np.NaN
draw_genus = np.NaN
draw_epithet = np.NaN
result_ims = []
df_list = []

for page_num in tqdm(index):
    #if page_num == index[-2]:
    #    break
    #page_num = index[-1]
    #process the pre-processed dfs
    page_df = df_dict[page_num]
    
    #for drawing
    pix_map = doc.get_page_pixmap(page_num,matrix=mat)
    image = Image.open(io.BytesIO(pix_map.tobytes()))
    draw = ImageDraw.Draw(image)

    #processing each column
    for c in page_df['col_no'].unique():
        col_df = page_df[page_df['col_no'] == c]
        col_df, genus, epithet, draw_genus, draw_epithet = process_col(col_df, genus, epithet, draw_genus, draw_epithet)
        df_list.append(col_df)

        #drawing boxes in each column
        plot_genus_blocks(col_df, draw)
        plot_epithet_blocks(col_df, draw)
        plot_author_blocks(col_df, draw)
        plot_infra_blocks(col_df, draw)

    result_ims.append(image)

#TIME_STR = datetime.now().strftime("%Y_%m_%d-%I_%M_%p")
result_ims[0].save(OUTPUT_PATH + 'vol1_index_ROI.pdf',save_all=True, append_images=result_ims[1:])

pre_processed_df = pd.concat([df_dict[k] for k in df_dict], axis = 0)
result_ims_valid_words[0].save(OUTPUT_PATH + "preprocessed/" + 'valid_words' + TAIL_STR + '.pdf',save_all=True, append_images=result_ims_valid_words[1:])
pre_processed_df.to_html(OUTPUT_PATH + "preprocessed/" + 'vol1_preprocessed_index' + TAIL_STR + '.html')
pre_processed_df.to_csv(OUTPUT_PATH + "preprocessed/" + 'vol1_preprocessed_index' + TAIL_STR + '.csv')

df = pd.concat(df_list, axis = 0)
df.to_html(OUTPUT_PATH + 'raw/' + 'vol1_index' + TAIL_STR + '.html')
df.to_csv(OUTPUT_PATH + 'raw/' + 'vol1_index' + TAIL_STR + '.csv', index = False)

pruned = df[(~df['genus'].isnull())]
pruned = pruned[["page_num", "genus", "epithet", "infra" ,"author", "taxon rank"]]
pruned.to_csv(OUTPUT_PATH + 'vol1_index_pruned' + TAIL_STR + '.csv', index = False)
pruned.to_html(OUTPUT_PATH + 'vol1_index_pruned' + TAIL_STR + '.html')

100%|██████████| 23/23 [00:30<00:00,  1.33s/it]


# YEET?

In [29]:
import fitz
import pprint, re

def flags_decomposer(flags):
    """Make font flags human readable."""
    l = []
    if flags & 2 ** 0:
        l.append("superscript")
    if flags & 2 ** 1:
        l.append("italic")
    if flags & 2 ** 2:
        l.append("serifed")
    else:
        l.append("sans")
    if flags & 2 ** 3:
        l.append("monospaced")
    else:
        l.append("proportional")
    if flags & 2 ** 4:
        l.append("bold")
    return ", ".join(l)

class Face:
    def __init__(self, font, size, color, flags=0):
        self.font = font
        self.size = size
        self.color = color
        self.flags = flags
        self.style = flags_decomposer(self.flags)
    def __str__(self):
        return f"Font: '{self.font}' ({self.style}), size {self.size:g}, color " + ("#%06x" % self.color)

In [30]:
def preprocessing(page_num, indent_err = 30):
    
    #initiate dataframe
    page_df = pd.DataFrame(pages[page_num].get_text_words(), columns =['in_x0', 'in_y0', 'in_x1', 'in_y1', 'word', 'block_no', 'line_no', 'word_no'])
    
    #add page number to dataframe
    page_df['page_num'] = np.array([page_num]*page_df.shape[0])
    #initiate all columns that will be added
    page_df['genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_genus'] = np.array([np.NaN]*page_df.shape[0])
    page_df['epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_epithet'] = np.array([np.NaN]*page_df.shape[0])
    page_df['author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_author'] = np.array([np.NaN]*page_df.shape[0])
    page_df['infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['draw_infra'] = np.array([np.NaN]*page_df.shape[0])
    page_df['taxon rank'] = np.array([np.NaN]*page_df.shape[0])
    page_df['error_check'] = np.array([np.NaN]*page_df.shape[0])
    #updating coordinates to represent target DPI
    page_df['x0'], page_df['y0'], page_df['x1'], page_df['y1'] = page_df['in_x0']*TARGET_DPI/ 72, page_df['in_y0']*TARGET_DPI/ 72, page_df['in_x1']*TARGET_DPI/ 72, page_df['in_y1']*TARGET_DPI/ 72
    #get x corner coordinates 
    x_min = page_df['x0'].min()
    x_max = page_df['x1'].max()

    y_max = page_df['y1'].max()

    #Remove the extra flore - 18 at page 545
    if page_num == index[4]:
        page_df = page_df[~((page_df["word"] == 'Flore') & (page_df['y1'] == y_max))]
    #invalid words dataframe -- for error checking
    pruned_words_df = page_df[~page_df["word"].apply(valid)].reset_index()

    #use this face thing to get the words  styles? 

    #prune out invalid words (based on function valid)
    #page_df = page_df[page_df["word"].apply(valid)].reset_index()
    
    indent_groups = []
    blocks = page_df['block_no'].unique()
    for b in blocks:
        lines = page_df[page_df['block_no'] == b]['line_no'].unique()
        for l in lines:
            #reset word_no values (useful for cases where word that was originally at 0th index was pruned out)
            cond = (page_df['line_no'] == l) & (page_df['block_no'] == b)
            num_words = len(page_df[cond]['word_no'])
            page_df.loc[cond, 'word_no'] = np.arange(num_words).astype(int) #this is slowww
            #set column number (0 or 1)
            x_0 = page_df[cond]['x0'].min()
            #THIS DOESN'T WORK AAAA -- issue was with line no thing
            if not np.isnan(x_0):
                page_df.loc[cond, 'col_no'] = np.array([int(x_0 > ((x_min + x_max) / 2))]*num_words).astype(int)

                #initiate indent groups -- only first word should get an indent_group value 
                new_group = True
                for g_i in range(len(indent_groups)):
                    g = indent_groups[g_i]
                    g_arr = np.array(g)
                    if x_0 <= np.mean(g_arr) + indent_err and x_0 >= np.mean(g_arr) - indent_err:
                        g.append(x_0)
                        new_group = False
                        page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)
                if new_group:
                    indent_groups.append([x_0])
                    g_i = len(indent_groups) - 1
                    page_df.loc[cond, 'indent_group'] = np.array([g_i]*num_words).astype(int)

    #print("indent groups:", indent_groups)
    #return updated page_df, pruned_words_df, indent groups
    return page_df.reset_index(), pruned_words_df, indent_groups

#https://stackoverflow.com/questions/53468558/adding-image-to-pandas-dataframe